# build-cat-ccam_noresm2-mm_historical_aus-10i_12km

In [1]:
import glob
import pathlib
import traceback
from datetime import datetime

import xarray as xr

from ecgtools import Builder
from ecgtools.builder import INVALID_ASSET, TRACEBACK

from matplotlib import pyplot as plt

# build the catalogue

In [2]:
root = '/g/data/xv83/mxt599/ccam_noresm2-mm_historical_aus-10i_12km/drs_cordex/CORDEX/output/AUS-10i/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1/'

In [3]:
files = sorted(glob.glob(root+'*/*/*'))
len(files)

32964

In [4]:
files[0]

'/g/data/xv83/mxt599/ccam_noresm2-mm_historical_aus-10i_12km/drs_cordex/CORDEX/output/AUS-10i/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1/1hr/CAPE/CAPE_AUS-10i_NCC-NorESM2-MM_historical_r1i1p1f1_CSIRO-CCAM-2203_v1_1hr_19510101-19511231.nc'

In [5]:
path = pathlib.Path(files[0])
path.stem.split('_')

['CAPE',
 'AUS-10i',
 'NCC-NorESM2-MM',
 'historical',
 'r1i1p1f1',
 'CSIRO-CCAM-2203',
 'v1',
 '1hr',
 '19510101-19511231']

# NOTES from Marcus chat

## use this as an error check?

# do we really need to open all the files?  Given variable name is in the filename?

In [6]:
def parse_CCAM(file):
    """CCAM data stored in"""
    file = pathlib.Path(file)
    info = {}

    try:
        stem = file.stem
        split = stem.split('_')
        variable = split[0]
        domain = split[1]
        host_GCM = split[2]
        run_type = split[3]
        host_ensemble = split[4]
        downscale_model = split[5]
        downscale_version = split[6]
        period = split[7]
        if period == '1hr':
            time_period = 'hourly'
        elif period == '6hr':
            time_period = 'six_hourly'
        elif period == 'day':
            time_period = 'daily'
        elif period == 'mon':
            time_period = 'monthly'
        else:
            time_period = 'fixed'    

        
        info = {
            'variable': variable,
            'domain': domain,
            'host_GCM':host_GCM,
            'run_type':run_type,
            'host_ensemble':host_ensemble,
            'downscale_model':downscale_model,
            'downscale_version':downscale_version,
            'period': period,
            'time_period': time_period,
            'path': str(file),
        }
        return info
    
    except Exception:
        return {INVALID_ASSET: file, TRACEBACK: traceback.format_exc()}

In [7]:
parse_CCAM(files[0])

{'variable': 'CAPE',
 'domain': 'AUS-10i',
 'host_GCM': 'NCC-NorESM2-MM',
 'run_type': 'historical',
 'host_ensemble': 'r1i1p1f1',
 'downscale_model': 'CSIRO-CCAM-2203',
 'downscale_version': 'v1',
 'period': '1hr',
 'time_period': 'hourly',
 'path': '/g/data/xv83/mxt599/ccam_noresm2-mm_historical_aus-10i_12km/drs_cordex/CORDEX/output/AUS-10i/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1/1hr/CAPE/CAPE_AUS-10i_NCC-NorESM2-MM_historical_r1i1p1f1_CSIRO-CCAM-2203_v1_1hr_19510101-19511231.nc'}

# setup builder object

In [8]:
root2 = '/g/data/xv83/mxt599/ccam_noresm2-mm_historical_aus-10i_12km/drs_cordex/CORDEX/output/AUS-10i/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1'

In [9]:
%%time
b = Builder([root2],depth=2)

CPU times: user 1.53 ms, sys: 0 ns, total: 1.53 ms
Wall time: 1.54 ms


In [10]:
b

Builder(paths=['/g/data/xv83/mxt599/ccam_noresm2-mm_historical_aus-10i_12km/drs_cordex/CORDEX/output/AUS-10i/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1'], storage_options={}, depth=2, exclude_patterns=[], include_patterns=[], joblib_parallel_kwargs={})

In [11]:
%%time
b.build(parsing_func = parse_CCAM)

CPU times: user 2.68 s, sys: 1.49 s, total: 4.17 s
Wall time: 8.78 s


/g/data/v14/tm4888/miniconda3/envs/intake-stable-ecgtools/lib/python3.10/site-packages/ecgtools/builder.py:204: UserWarning: Unable to parse 4 assets. A list of these assets can be found in `.invalid_assets` attribute.
  ).clean_dataframe()


Builder(paths=['/g/data/xv83/mxt599/ccam_noresm2-mm_historical_aus-10i_12km/drs_cordex/CORDEX/output/AUS-10i/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1'], storage_options={}, depth=2, exclude_patterns=[], include_patterns=[], joblib_parallel_kwargs={})

In [12]:
b.df

,variable,domain,host_GCM,run_type,host_ensemble,downscale_model,downscale_version,period,time_period,path
0,CAPE,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,1hr,hourly,/g/data/xv83/mxt599/ccam_noresm2-mm_historical...
1,CAPE,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,1hr,hourly,/g/data/xv83/mxt599/ccam_noresm2-mm_historical...
2,CAPE,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,1hr,hourly,/g/data/xv83/mxt599/ccam_noresm2-mm_historical...
3,CAPE,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,1hr,hourly,/g/data/xv83/mxt599/ccam_noresm2-mm_historical...
4,CAPE,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,1hr,hourly,/g/data/xv83/mxt599/ccam_noresm2-mm_historical...
...,...,...,...,...,...,...,...,...,...,...
32959,zmla,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,mon,monthly,/g/data/xv83/mxt599/ccam_noresm2-mm_historical...
32960,zmla,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,mon,monthly,/g/data/xv83/mxt599/ccam_noresm2-mm_historical...
32961,zmla,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,mon,monthly,/g/data/xv83/mxt599/ccam_noresm2-mm_historical...
32962,zmla,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,mon,monthly,/g/data/xv83/mxt599/ccam_noresm2-mm_historical...


In [13]:
dir(b)

['__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_init_post_parse__',
 '__pydantic_initialised__',
 '__pydantic_model__',
 '__pydantic_run_validation__',
 '__pydantic_validate_values__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__validate__',
 '__weakref__',
 '_root_dirs',
 'assets',
 'build',
 'clean_dataframe',
 'depth',
 'df',
 'entries',
 'exclude_patterns',
 'exclude_regex',
 'get_assets',
 'include_patterns',
 'include_regex',
 'invalid_assets',
 'joblib_parallel_kwargs',
 'parse',
 'paths',
 'save',
 'storage_options']

In [14]:
b.invalid_assets['INVALID_ASSET'].values

array([PosixPath('/g/data/xv83/mxt599/ccam_noresm2-mm_historical_aus-10i_12km/drs_cordex/CORDEX/output/AUS-10i/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1/fx/orog/orog_AUS-10i_NCC-NorESM2-MM_historical_r1i1p1f1_CSIRO-CCAM-2203_v1.nc'),
       PosixPath('/g/data/xv83/mxt599/ccam_noresm2-mm_historical_aus-10i_12km/drs_cordex/CORDEX/output/AUS-10i/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1/fx/sftlaf/sftlaf_AUS-10i_NCC-NorESM2-MM_historical_r1i1p1f1_CSIRO-CCAM-2203_v1.nc'),
       PosixPath('/g/data/xv83/mxt599/ccam_noresm2-mm_historical_aus-10i_12km/drs_cordex/CORDEX/output/AUS-10i/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1/fx/sftlf/sftlf_AUS-10i_NCC-NorESM2-MM_historical_r1i1p1f1_CSIRO-CCAM-2203_v1.nc'),
       PosixPath('/g/data/xv83/mxt599/ccam_noresm2-mm_historical_aus-10i_12km/drs_cordex/CORDEX/output/AUS-10i/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1/fx/sfturf/sfturf_AUS-10i_NCC-NorESM2-MM_historical_r1i1p1f1_CSIRO-C

## example from docs

In [15]:
!pwd -P

/home/599/tm4888


In [17]:
b.save(
    # File path - could save as .csv (uncompressed csv) or .csv.gz (compressed csv)
    name = "/intake-catalogues/hist-01052023-catalogue",
    # Column name including filepath
    path_column_name='path',
    # Column name including variables
    variable_column_name='variable',
    # Data file format - could be netcdf or zarr (in this case, netcdf)
    data_format="netcdf",
    # Which attributes to groupby when reading in variables using intake-esm
    groupby_attrs=["domain", "host_GCM", "run_type","host_ensemble","downscale_model","downscale_version","period"],
    # Aggregations which are fed into xarray when reading in data using intake
    aggregations=[
        {
            "type": "join_existing",
            "attribute_name": "date",
            "options": {"dim": "time", "coords": "minimal", "compat": "override"},
        }
    ],
)

Successfully wrote ESM catalog json file to: file:///home/599/tm4888//intake-catalogues/hist-01052023-catalogue.json


/jobfs/82277945.gadi-pbs/ipykernel_3344215/992739043.py:1: UserWarning: Unable to parse 4 assets/files. A list of these assets can be found in /jobfs/82277945.gadi-pbs//intake-catalogues/hist-01052023-catalogue_invalid_assets.csv.
  b.save(


OSError: Cannot save file into a non-existent directory: '/jobfs/82277945.gadi-pbs/intake-catalogues'

# Can I querry the catalogue? YES!

In [ ]:
import intake

### SICK OF ALL THESE WARNINGS?

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data_catalog = intake.open_esm_datastore('/g/data/v14/tm4888/code/ACS/data-catalogue/catalogues/test-CCAM-catalogue.json')
data_catalog

In [ ]:
data_catalog.df

In [ ]:
data_catalog.df.variable.unique()

In [ ]:
data_catalog.df.period.unique()

In [ ]:
search1 = data_catalog.search(variable=['tas'],period='mon')
search1

In [ ]:
search1.df

In [ ]:
%%time
dsets = search1.to_dataset_dict()

In [ ]:
dsets.keys()

In [ ]:
search_DS = dsets['AUS-10i.NCC-NorESM2-MM.historical.r1i1p1f1.CSIRO-CCAM-2203.v1.mon']

In [ ]:
search_DS

In [ ]:
mean_tas = search_DS.tas.mean(dim='time')
mean_tas

In [ ]:
%%time
mean_tas = mean_tas.compute()

In [ ]:
mean_temp = mean_tas - 273.15

In [ ]:
mean_temp.plot(robust=True)
plt.title('Mean Near-Surface Air Temperature in degrees C for \n AUS-10i.NCC-NorESM2-MM.historical.r1i1p1f1.CSIRO-CCAM-2203.v1.mon')

# seasonal climatology

In [ ]:
tas_season_mean = search_DS.tas.groupby('time.season').mean('time')

In [ ]:
tas_season_mean

In [ ]:
temp_season_mean = tas_season_mean-273.15

In [ ]:
temp_season_mean

In [ ]:
temp_season_mean.sel(season='JJA').plot(robust=True)
plt.title('Winter(JJA) Near-Surface Air Temperature in degrees C for \n AUS-10i.NCC-NorESM2-MM.historical.r1i1p1f1.CSIRO-CCAM-2203.v1.mon')

In [ ]:
client.close()